# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [3]:
# Read the Mapbox API key
load_dotenv()
os.getenv("MAPBOX_API_KEY")

# Set the Mapbox API
px.set_mapbox_access_token(map_box_api)


# Import Data

In [4]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [5]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    minimum_units = sfo_data["housing_units"].min()
    maximum_units = sfo_data["housing_units"].max()
    average_units = sfo_data["housing_units"].groupby(['year']).mean()
    average_units_plot = average_units.plot.bar(title = "Housing Units in San Francisco from 2010 to 2016")
    average_units_plot.set_ylabel("Housing Units")
    average_units_plot.set_xlabel("Year")
    plt.ylim(minimum_units, maximum_units)
    plt.show()

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    average_rent_sales = sfo_data.groupby(['year']).mean()
    average_rent_sales = average_rent_sales.drop(columns=['housing_units'])
    average_rent_sales_plot = average_rent_sales["gross_rent"].plot(title = "Average Gross Rent in San Francisco")
    average_rent_sales_plot.set_ylabel("Gross Rent")
    average_rent_sales_plot.set_xlabel("Year")
    plt.show()

def average_sales_price():
    """Average Sales Price Per Year."""
    average_rent_sales = sfo_data.groupby(['year']).mean()
    average_rent_sales = average_rent_sales.drop(columns=['housing_units'])
    rent_plot = average_rent_sales["sale_price_sqr_foot"].plot(title="Average Sale Price per Square Foot in San Francisco")
    rent_plot.set_ylabel("Avg. Sale Price")
    rent_plot.set_xlabel("Year")
    plt.show()
    
def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    average_neighborhood2 = sfo_data.groupby(['year', 'neighborhood']).mean()
    return average_neighborhood2.loc[:, 'sale_price_sqr_foot'].hvplot(title = 'Average Price by Neighborhood', groupby = 'neighborhood', rot = 90, height = 400)
        
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    top_ten = sfo_data.groupby('neighborhood').mean()
    top_ten = top_ten.sort_values(by='sale_price_sqr_foot', ascending=False).head(10)
    top_ten_plot = top_ten.loc[:, 'sale_price_sqr_foot'].hvplot.bar(title = "Top 10 Most Expensive Neighborhoods in San Francisco", rot = 90, height = 400)
    return top_ten_plot
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    average_neighborhood = sfo_data.groupby(['neighborhood']).mean()
    average_neighborhood = average_neighborhood.reset_index()
    return px.parallel_coordinates(average_neighborhood, color = "sale_price_sqr_foot", color_continuous_scale = px.colors.sequential.Inferno)

def parallel_categories():
    """Parallel Categories Plot."""
    top_ten = sfo_data.groupby('neighborhood').mean()
    top_ten = top_ten.sort_values(by='sale_price_sqr_foot', ascending=False).head(10)
    return px.parallel_categories(
    top_ten,
    dimensions=["sale_price_sqr_foot", "housing_units", "gross_rent"],
    color="sale_price_sqr_foot",
)

def neighborhood_map():
    """Neighborhood Map"""
    mean_values = sfo_data.groupby(['neighborhood']).mean()
    joined = df_neighborhood_locations.join(mean_values, on = 'Neighborhood')
    joined = joined.dropna()
    return px.scatter_mapbox(
    joined,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    zoom=4
    )
    

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [1]:
row2 = pn.Column(housing_units_per_year(), average_gross_rent(), average_sales_price())
row3 = pn.Column(average_price_by_neighborhood(), top_most_expensive_neighborhoods())
row4 = pn.Row(parallel_coordinates, parallel_categories)

In [7]:
# Create tabs
panel = pn.Tabs(
    ("Welcome", neighborhood_map), ("Yearly Market Analysis", row2), ("Neighborhood Analysis", row3), ("Parallel Plots Analysis", row4))

## Serve the Panel Dashboard

In [8]:
panel.servable()

Tabs
    [0] Column
        [0] Column()
        [1] Row
            [0] Plotly(Figure, name='interactive01592')
    [1] Column
        [0] Str(None)
        [1] Str(None)
        [2] Str(None)
    [2] Column
        [0] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                [1] VSpacer()
        [1] HoloViews(Bars)
    [3] Row
        [0] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01579')
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01585')